In [1]:
# Author: Muhammad Tahir A.

In [2]:
import sys 
import json
import time
import pyodbc
import logging
import petl as etl
import pandas as pd
from datetime import datetime
from requests import get,post
import pprint

from general_helper import *
from connection import *
from load_env_var import *

In [3]:
tradbDev_conn = mongodb_connection(tradbDev_str,tradbDev_db)
traDataDump_conn = mongodb_connection(traDataDump_str,traDataDump_db)

In [4]:
file = "TRA-Monday-Report-Mar-8-2021-Mar-14-2021.xlsx"
fb_costs = pd.read_excel(file)
print(len(fb_costs))

242


In [5]:
REQUIRED_COLUMNS = [['Campaign Name', 'CampaignId', 'adsetid', 'adid', 'Day', 'AdSetName', 'AdName', 
                     'social_reach', 'impressions', 'frequency', 'Result Type', 'Results', 'Cost per Result', 
                     'spent', 'cpc', 'page_likes', 'post_engagement', 'video_views', 'cost_per_lead_lp', 
                     'leads_form', 'clicks',
                     'dim_dateid', 'CorpWeekId', 'dim_participantid', 'dimProgramId']]
                     
PARTICIPANT = 'Good Sam'

In [6]:
fb_costs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 242 entries, 0 to 241
Data columns (total 20 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Campaign Name       242 non-null    object 
 1   Ad Set Name         242 non-null    object 
 2   Ad Name             242 non-null    object 
 3   Campaign ID         242 non-null    int64  
 4   Ad Set ID           242 non-null    int64  
 5   Ad ID               242 non-null    int64  
 6   Day                 242 non-null    object 
 7   Delivery Status     242 non-null    object 
 8   Delivery Level      242 non-null    object 
 9   Result Type         242 non-null    object 
 10  Results             117 non-null    float64
 11  Reach               242 non-null    int64  
 12  Impressions         242 non-null    int64  
 13  Cost per Result     117 non-null    float64
 14  Amount Spent (USD)  242 non-null    float64
 15  Frequency           242 non-null    float64
 16  Clicks (

In [7]:
fb_costs.head()

,Campaign Name,Ad Set Name,Ad Name,Campaign ID,Ad Set ID,Ad ID,Day,Delivery Status,Delivery Level,Result Type,Results,Reach,Impressions,Cost per Result,Amount Spent (USD),Frequency,Clicks (All),CTR (All),Reporting Starts,Reporting Ends
0,#692: TRA - North Shore Landing (Lead Ad) 8.20,INT,North Shore Landing - Trailer/Trees - ID 277,23845724443430001,23845724443330001,23845724443410001,2021-03-14,active,ad,On-Facebook Leads,18.0,6814,7522,5.353333,96.36,1.103904,236,3.137463,2021-03-14,2021-03-14
1,#379: TRA - Sycamore Lodge (Lead Ad) 8.20,LAL,Sycamore Lodge - Spring Class A - ID 278,23845724456710001,23845724456720001,23847034717640001,2021-03-14,active,ad,On-Facebook Leads,1.0,538,578,8.640000,8.64,1.074349,20,3.460208,2021-03-14,2021-03-14
2,#1566: TRA - Rocky Fork Ranch (Lead Ad) 10.20,INT,Rocky Fork Ranch - Tent - ID 262,23846228223300001,23846228415760001,23847301634380001,2021-03-14,active,ad,On-Facebook Leads,NaN,32,43,NaN,0.42,1.343750,1,2.325581,2021-03-14,2021-03-14
3,#692: TRA - North Shore Landing (Lead Ad) 8.20,LAL,North Shore Landing - Train Cabin - ID 277,23845724443430001,23845724443420001,23846777220670001,2021-03-14,active,ad,On-Facebook Leads,NaN,159,167,NaN,2.58,1.050314,10,5.988024,2021-03-14,2021-03-14
4,#1566: TRA - Rocky Fork Ranch (Lead Ad) 10.20,CA,Rocky Fork Ranch - Cabins - ID 262,23846228223300001,23846228223330001,23846228359320001,2021-03-14,active,ad,On-Facebook Leads,NaN,171,188,NaN,2.07,1.099415,3,1.595745,2021-03-14,2021-03-14


In [8]:
fb_costs.columns

Index(['Campaign Name', 'Ad Set Name', 'Ad Name', 'Campaign ID', 'Ad Set ID',
       'Ad ID', 'Day', 'Delivery Status', 'Delivery Level', 'Result Type',
       'Results', 'Reach', 'Impressions', 'Cost per Result',
       'Amount Spent (USD)', 'Frequency', 'Clicks (All)', 'CTR (All)',
       'Reporting Starts', 'Reporting Ends'],
      dtype='object')

In [9]:
print(len(fb_costs), 'Number of Rows in file.')
if len(fb_costs) == 0:
    print('0 Rows found in file, Continue to next file.')

fb_costs = fb_costs.rename(columns={
    'Campaign ID': 'CampaignId', 'Ad Set ID': 'adsetid', 'Ad set ID': 'adsetid',
    'Ad ID': 'adid', 'Ad Set Name': 'AdSetName', 'Ad set name': 'AdSetName',
    'Ad Name': 'AdName', 'Ad name': 'AdName', 'Reach': 'social_reach',
    'Impressions': 'impressions','Frequency':'frequency',
    'Amount Spent (USD)': 'spent', 'Amount spent (USD)': 'spent','CPC (All)': 'cpc',
    'CPC (all)': 'cpc','Page Likes': 'page_likes', 
    'Post Engagement': 'post_engagement', 'Post engagement': 'post_engagement', 'Video Plays': 'video_views',
    'Video plays': 'video_views', 'Cost per Lead': 'cost_per_lead_lp', 'Page likes': 'page_likes', 
    'Leads (Form)': 'leads_form', 'Leads (form)': 'leads_form', 'Clicks (All)': 'clicks', 'Clicks (all)': 'clicks', 
    'Campaign Name': 'CampaignName','Campaign name': 'CampaignName'})

# Converting date string into datetime object
fb_costs['Day'] = pd.to_datetime(fb_costs['Day'])

# Converting datetime object to Corp Week Id
corp_weeks = json.load(open('corp_weeks.json'))
fb_costs['CorpWeek'] = fb_costs[['Day']] \
    .apply(lambda x: corp_weeks.get(str(x[0])[:10], 0), axis=1)

# Attaching Corp Week Id using Dim_CorpWeek table in warehouse

fb_costs['date'] = fb_costs[['Day']] \
    .apply(lambda x: str(x[0])[:10], axis=1)

REQUIRED_COLUMNS = ['CampaignName', 'CampaignId', 'adsetid', 'adid', 'AdSetName', 'AdName',
    'social_reach', 'impressions', 'frequency', 'spent',
    'clicks','CorpWeek','date']

fb_costs = fb_costs[REQUIRED_COLUMNS]

fb_costs['dim_participantid'] = 7430
fb_costs['dimProgramId'] = 1000000195
fb_costs['CampaignManagerId'] = fb_costs[['AdName']]  \
    .apply(lambda x: get_campaign_manager_id_from_adname(x[0]), axis=1)

fb_costs['resort'] = ''

for i in ['CampaignName', 'adsetid', 'adid', 'AdSetName', 'AdName', 'resort']:
    fb_costs[i] = fb_costs[i].fillna('').apply(str)

for i in ['social_reach', 'impressions', 
        'clicks', 'CampaignManagerId']: # 'CorpWeekId', 'dim_dateid', 'dim_participantid', 'dimProgramId'
    fb_costs[i] = fb_costs[i].fillna(0).replace('', 0).apply(int)

for i in ['frequency', 'spent']:
    fb_costs[i] = fb_costs[i].fillna(0.0).replace('', 0.0).apply(float)

# decoding ascii's
# fb_costs['AdSetName'] = fb_costs[['AdSetName']] \
#     .apply(lambda x: encode_agent_name(x[0]).title(), axis=1)


242 Number of Rows in file.


In [10]:
fb_costs.columns

Index(['CampaignName', 'CampaignId', 'adsetid', 'adid', 'AdSetName', 'AdName',
       'social_reach', 'impressions', 'frequency', 'spent', 'clicks',
       'CorpWeek', 'date', 'dim_participantid', 'dimProgramId',
       'CampaignManagerId', 'resort'],
      dtype='object')

In [11]:
fb_costs.head(1)

,CampaignName,CampaignId,adsetid,adid,AdSetName,AdName,social_reach,impressions,frequency,spent,clicks,CorpWeek,date,dim_participantid,dimProgramId,CampaignManagerId,resort
0,#692: TRA - North Shore Landing (Lead Ad) 8.20,23845724443430001,23845724443330001,23845724443410001,INT,North Shore Landing - Trailer/Trees - ID 277,6814,7522,1.103904,96.36,236,3510,2021-03-14,7430,1000000195,277,


In [12]:
tags_collection=traDataDump_conn["tags"]
adsSpent_collection = tradbDev_conn["spentData"]

docs_container=[]
for v in fb_costs.itertuples():
    spentDataPayload={}
    spentDataPayload["contactAttempts"]=getTradbTags(tags_collection,int(v.CampaignManagerId))
    spentDataPayload["campaign"]=v.CampaignId
    spentDataPayload["spent"]= calculate_spent(v.spent,0.25)
    spentDataPayload["LeadCorpWeek"]=v.CorpWeek
    spentDataPayload["dateCreated"]=v.date
    spentDataPayload["type"]= "spend"
    spentDataPayload["campaignManagerId"]=v.CampaignManagerId
    spentDataPayload["subtype"]="service_charges" 
    pprint.pprint(spentDataPayload)
    docs_container.append(spentDataPayload)
    break

print(mongodb_many_to_many_insert(adsSpent_collection,docs_container))

{'LeadCorpWeek': 3510,
 'campaign': 23845724443430001,
 'campaignManagerId': 277,
 'contactAttempts': {'campaignManagerId': 277,
                     'campaignType': 'Five9 Outbound',
                     'contactType': 'Five9 Outbound',
                     'dnis': '0',
                     'isActive': True,
                     'name': 'Good Sam Facebook Campaign',
                     'promotion': '4 DAYS 3 NIGHTS FREE CAMPING',
                     'resort': ObjectId('5d52cde18bb76e30d8ef53f6'),
                     'tag_mongoId': '5f468d353ab2d58d3b4ebffc'},
 'dateCreated': '2021-03-14',
 'spent': 24.09,
 'subtype': 'service_charges',
 'type': 'spend'}
{'acknowledged': True, 'inserted_records': 1, 'records': 1}


In [ ]:
# from ftplib import FTP

# ftp = FTP('tmidev.net', 'tmidev', 'qwer4321!@#') 
# ftp.cwd('/fbreports/goodsam')
# ftp.nlst()

# files_on_ftp = ftp.nlst()
# # Cleaning existing file lists

# try:
#     files_on_ftp.remove('.')
#     files_on_ftp.remove('..')
# except:
#     pass

# files_on_ftp
#ftp.quit()

In [ ]:
# Inserting file log into warehous
# cursor = dw_conn.cursor()
# query = """
# INSERT INTO Fact_FbCostFilesDev VALUES('{}', '{}', '{}')
# """.format(filename, PARTICIPANT, str(datetime.now())[:10])
# cursor.execute(query)
# dw_conn.commit()

# test = fb_costs.loc[
#     fb_costs.CorpWeekId == 0
# ]

# if len(test) > 0:
#     print 'Got 0 in CorpWeekId,', filename

# table = etl.fromdataframe(fb_costs)
# etl.io.appenddb(table, dw_conn, 'Fact_FbCampaignDev')

In [ ]:
# existing_files = pd.read_sql("""
# SELECT * FROM fact_fbCostFilesDev where Participant='{}'
# """.format(PARTICIPANT), dw_conn)

# print len(existing_files)
# existing_files = list(existing_files.filename.unique())

# for filename in files_on_ftp:
    
    # if filename in existing_files:
    #     # Old file
    #     continue
    
    # print 'Downloading', filename
    
    # ftp.retrbinary('RETR ' + filename, open(filename, 'wb').write)
    # time.sleep(1)
    # fb_costs = pd.read_excel(filename, 
    #                        dtype={'Campaign ID':str,'Ad Set ID':str, 'Ad ID': str}).fillna('')
    